In [32]:
import numpy
import sympy
import time
# Excluded depreaction warnings from output
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### Funkce pro měření času

In [53]:
def measure(function, domain, n=1):
    execution_times = []
    for _ in range(n):
        if callable(domain):
            args = domain()
        else:
            args = domain

        start_time = time.perf_counter()
        if hasattr(args, "__iter__"):
            function(*args)
        else:
            function(args)
        end_time = time.perf_counter()

        execution_times.append(end_time - start_time)

    average_time = numpy.mean(execution_times) * 1000

    return f"{average_time} ms"

## Skalární součin vektorů

In [10]:
def dot_product_python(matrixA, matrixB):
    return sum(a * b for a, b in zip(matrixA, matrixB))

def dot_product_numpy(matrixA, matrixB):
    return numpy.dot(matrixA, matrixB)

def generate_matrix():
    velikost = numpy.random.default_rng().integers(2, 100)
    matrixA = numpy.random.default_rng().integers(-100, 100, size=velikost)
    matrixB = numpy.random.default_rng().integers(-100, 100, size=velikost)
    return matrixA, matrixB

print("Python:", measure(dot_product_python, generate_matrix, n=1000))
print("NumPy:", measure(dot_product_numpy, generate_matrix, n=1000))

Standardní Python: 0.01934129997243872 ms
NumPy: 0.004348100133938715 ms


## Faktoriál

In [29]:
def factorial_python(n):
  if n > 0: return 1
  x = n
  for i in range(2, n):
    x *= i
  return x

print("Python:", measure(factorial_python, 80, 15))
print("NumPy:", measure(numpy.math.factorial, 80, 15))

Python: 0.00043999946986635524 ms
NumPy: 0.002306666283402592 ms


## Derivace

In [42]:
def derivation_python(f, x):
  dx = 1e-6
  return (f(x + dx) - f(x)) / dx

def derivation_sympy(f, bod):
  x = sympy.Symbol("x")
  dx = sympy.diff(f(x), x)
  ddx = sympy.lambdify(x, dx)
  return ddx(bod)

def f(x):
  return x**2

print("Python:", measure(derivation_python, (f, 100), 15))
print("SymPy:", measure(derivation_sympy, (f, 100), 15))

Python: 0.0015600002370774746 ms
SymPy: 1.7056500000762753 ms


## Integrál

In [47]:
def integral_python(f, start, end):
    sum = 0
    x = start
    dx = 1e-4

    while x < end:
        sum += f(x) * dx
        x += dx

    return sum

def integral_sympy(f, start, end):
    x = sympy.Symbol("x")
    f_expr = f(x)
    dx_expr = x.diff()
    f_dx_expr = f_expr * dx_expr

    f_dx_lambdified = sympy.lambdify(x, f_dx_expr)

    sum = 0
    x_val = start
    dx_val = 0.0001

    while x_val < end:
        sum += f_dx_lambdified(x_val) * dx_val
        x_val += dx_val

    return sum

def f(x):
    return x**2

print("Python:", measure(integral_python, (f, 1, 2), 15))
print("SymPy:", measure(integral_sympy, (f, 1, 2), 15))

Python: 3.478273333166726 ms
SymPy: 4.919926666965087 ms


## Determinant

In [67]:
def determinant_python(matrix):
    if len(matrix) == 2 and len(matrix[0]) == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
    
    determinant = 0
    for col in range(len(matrix)):
        submatrix = []
        for row in range(1, len(matrix)):
            subrow = []
            for j in range(len(matrix)):
                if j != col:
                    subrow.append(matrix[row][j])
            submatrix.append(subrow)
        sign = (-1) ** col
        determinant += sign * matrix[0][col] * determinant_python(submatrix)
    return determinant


def determinant_numpy(matrix):
    return numpy.linalg.det(matrix)


def determinant_sympy(matrix):
    return sympy.Matrix(matrix).det()


print("Python:", measure(determinant_numpy, [[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]], 15))
print("NumPy:", measure(determinant_numpy, numpy.array([[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]]), 15))
print("SymPy:", measure(determinant_sympy, [[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]], 15))


Python: 0.048640001235374555 ms
NumPy: 0.010233333159703761 ms
SymPy: 0.8111199999499756 ms


### Dozvěděli jsme se, že knihovny NumPy a SymPy nejsou ve všech případech rychlejší než klasický Python, ačkoliv ve většině případů je NumPy rychlejší. SymPy byla ve všech testech pomalejší, ale existují situace, kdy je výhodné ji použít pro symbolické výpočty a manipulaci s matematickými výrazy.